In [10]:
import requests
import json
from dotenv import load_dotenv
from azure.identity import ClientSecretCredential
import random
import pandas as pd
import numpy as np
import os
import time

load_dotenv('testing.env')

True

In [2]:
def get_token():

    credential = ClientSecretCredential(tenant_id=os.environ['openai_tenant_id'],
                                    client_id=os.environ['client_id'],
                                    client_secret=os.environ['client_secret'])
    jwt = credential.get_token(os.environ['scope'])
    token = jwt.token
    
    return token

In [3]:
# Each user waits 10 seconds between responses
# 5 concurrent users (from one BU)

# 1 request every ~2 + processing time (~3) seconds = 5 seconds 

# How many requests are ok in 1 minute? depends on length of story.

# Get an estimate for BAI and QAI

In [4]:
stories = ['''As a NC System, when document of any type is received from TMNA, the document will be persisted, if the distCd is present and match 91041, (Mandatory) 

As a NC System, when document of any type is received from TMNA, the document will be ignored, if the distCd is present and not match 91041,

As a NC System, when document of any type is received from TMNA, the document will be ignored, if the distCd is not present and IgnoreMissingDistCodeDocs = true (Default = True) 

As a NC System, when document of any type is received from TMNA, the document will be persisted, if the distCd is not present and IgnoreMissingDistCodeDocs = false ''',
'''Perform analysis to come up with the requirements needed to restructure the way eWorkOrder handles authorization.

Currently authorization is managed by roles which are hard coded within the mobile app to specfic features.
For example, Supervisor role can access features:  A,C,D,E but Driver can only access A and E (just fake examples).

Confirm the new requirements by documenting the new requirements along with what needs to happen to make such a transition.  Some questions to ask would be:

Should we redefine roles to be per feature?
Should we keep the existing roles but make them dynamic such that the feature assignment of a role is based on a database collection? 
We can consider other roles beyond just access to a feature, such as FeatureA_Readonly and/or FeatureA_DriveOnly
Once the implementation is restructured, how do we deal with placing users into the new roles?  Is there an automated way (script in which ITSAccess can come up with?)

Database collection approach:

Imagine this is a record in the database.  At any given time we could add or remove tokens from the below.  For example, imagine we want to disable work order printing for this role, we would just remove the token VehiclePrintWorkOrderButton from below.  The mobile app could refresh the rules either between login sessions or every time a user comes back into the app from background to foreground.

"set_eworkorder_arrival_users":[
                    "RailArrivalsView"
                    "ShipArrivalsView"
                    "VehicleDriveButtonProd"
                    "VehiclePrintWorkOrderButton"
                    "FindVehicleView"
                ]
''',
'''In order to prevent pre-payment cancellations from being sent to NetSuite
the team
needs to prevent Vendor Invoices with a negative amount and an INVOICETYPE of "VendorAdvance" from being sent to Netsuite

Prepayments are sent to NetSuite as a Vendor Invoice.  Those work correctly.  
When one of those prepayments need to be cancelled, it will first be cancelled in NetSuite.  
Then, the prepayemnt will be cancelled in D365.  This will create a negative vendor invoice and will be sent out of D365.
Since this prepayment has already been cancelled in NetSuite, we do NOT want the D365 record sent to NetSuite.  

Prevent the following vendor Invoice type from going to NetSuite
 - INVOICETYPE = VendorAdvance
  - Amount is negative 

This story was created in response to Defect 401104''',
'''As an eWO service advocate , I want to restrict user access to eWO web functions based on the AD group to app token configuration so that users will be presented with menu options that he is configured to use '''
]

In [5]:
request_template = {
  "chat_history": [
    {
      "role": "user",
      "content": "As a parts analyst, I need item velocity code rules added for p-coded parts So that the P-Coded parts are not removed from stocking during item eligibility updates in the P&F system. #validate",
      "resources": []
    }
  ],
  "enabler" : False,
  "enabler_type": "Spike"
}

In [26]:
responses = []
request_count = 0
successes = 0
times = []
while sum(times) < 120:
    start = time.time()
    token = get_token()
    request_body = request_template.copy()
    request_body['chat_history'][0]['content'] = random.sample(stories, 1)[0]
    request_body['enabler'] = random.sample([True, False ], 1)[0]
    request_count += 1
    resp = requests.post(os.environ['apim_jma_url'] + 'requirementsBai',
                                json.dumps(request_body),
                                headers={'Authorization': f'Bearer {token}'})
    end = time.time()
    times.append(end - start)
    responses.append(resp.json())
    print(resp.status_code)
    if resp.status_code == 200:
        successes += 1
    # time.sleep(2)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [27]:
successes

23

In [181]:
def test_endpoint(code, requirements = None, unit_test_framework = None):

    body = { 
    "unit_test_input": 
        {
            "code": code,
            "requirements": requirements,
            "unit_test_framework": unit_test_framework
            
        }
    }

    response = requests.post('http://localhost:7071/api/unitTestQai', json.dumps(body))
    response = response.json()
    response = response['response']['content']

    return response
    

In [3]:
code = '''def get_warranty_details_with_exception_check(make, model, year, cosmos_db):
    container = cosmos_db.get_container_client(os.environ['cosmos_warranty_container'])

    model_upper = model.upper() if model else ""

    query = (f"SELECT * FROM c WHERE UPPER(c.Make) = '{make.upper() if make else ''}' AND {year} >= c.MYStart "
             f"AND ({year} <= c.MYEnd OR ISNULL(c.MYEnd)) "
             f"AND (CONTAINS(UPPER(c.Model), '{model_upper}') OR ISNULL(c.Model) OR c.Model = '')"
             f"ORDER BY c.Model DESC")
    items = list(container.query_items(query=query, enable_cross_partition_query=True))

    make_year = f'Make: {make}, Model: {model}, Year: {year}'
    logging.info(f'{make_year}')
    logging.info(f"Number of items found: {len(items)}")
    logging.info(f"Items found: {items}")

    if items:  # Checks if the list is not empty
        if model_upper:
            item = items[0]  # Get the first item if model_upper is not empty
        else:
            item = items[-1]  # Get the last item if model_upper is empty

        # Check if the model of the selected item is in the list of specified models
        item_model_upper = item.get('Model', '').upper() if item.get('Model') else ''
        if any(m in item_model_upper for m in ["TAHOE", "SUBURBAN", "PICKUP", "SIERRA", "YUKON"]):
            raise NotEligibleForFCheck('Not Eligible for Manufacturer Factory Warranty', {'reason': 'Factory Inconclusive'})
            
        has_exception = item.get('Exception') is not None
        return has_exception, item  # Return the exception flag and the single item
    
        # if fuel_type.upper() == "HYBRID" and model_upper == "HYBRID":
        #     # Filter items to find the record with Model "Hybrid"
        #     items = [item for item in items if item.get('Model', "").upper() == "HYBRID"]
    else:
        # If no items are found, raise VehicleUnavailable exception
        raise VehicleUnavailable('Vehicle Not Found', {"Vehicle": make_year})'''

In [4]:
requirements = "As a Mechanical Claims Analyst,\nI want to implement handling logic for manufacturer's warranty exceptions,\nSo that the correct warranty  was 10/100,000.\n2017 and newer Nissan Titan has a 5/100,000 bumper-to-bumper warranty.\n2019 - 2018 VW e-Golf was excluded from Volkswagen's 6-year warranty.\nRaider has a 5/60,000 full warranty with no standard extension for the Powertrain.\n2004-2023 Lancer Ralliart has a 3/36,000 full warranty and a 5/60,000 powertrain warranty.\nScope:\nNew Vehicles: In-Service Miles  500 miles\nNotes:\nThis user story focuses on implementing exception handling for the warranty terms.\nThe acceptance criteria specify the warranty coverage for both Bumper-to-Bumper and Powertrain Limited Warranty for this specific vehicle model within the defined model year range.\nException handling ensures accurate application of warranty terms for the Vehicle, enhancing claim processing efficiency and providing clarity on warranty coverage for this particular vehicle.\nAcceptance criteria:\nGiven a vehicle is known to have a manufacturer's warranty exception,\nWhen validating if the vehicle is covered under the manufacturer's warranty,\nThen the correct warranty thresholds are applied to the vehicle."

print(requirements)

As a Mechanical Claims Analyst,
I want to implement handling logic for manufacturer's warranty exceptions,
So that the correct warranty  was 10/100,000.
2017 and newer Nissan Titan has a 5/100,000 bumper-to-bumper warranty.
2019 - 2018 VW e-Golf was excluded from Volkswagen's 6-year warranty.
Raider has a 5/60,000 full warranty with no standard extension for the Powertrain.
2004-2023 Lancer Ralliart has a 3/36,000 full warranty and a 5/60,000 powertrain warranty.
Scope:
New Vehicles: In-Service Miles  500 miles
Notes:
This user story focuses on implementing exception handling for the warranty terms.
The acceptance criteria specify the warranty coverage for both Bumper-to-Bumper and Powertrain Limited Warranty for this specific vehicle model within the defined model year range.
Exception handling ensures accurate application of warranty terms for the Vehicle, enhancing claim processing efficiency and providing clarity on warranty coverage for this particular vehicle.
Acceptance criteria

In [ ]:
print('')

In [5]:
response = test_endpoint(code, requirements, unit_test_framework = "pytest")

In [6]:
print(response['test_plan'])

1. When connecting to CosmosDB:
    - Ensure `cosmos_db` argument is not `None` and is a valid client object
    - Check if the CosmosDB container specified in `os.environ['cosmos_warranty_container']` exists
2. When querying for vehicle warranty items: 
    - Ensure vehicles with correct `make`, `model`, `year` arguments are retrieved
    - Handle `None`, empty string or invalid inputs gracefully 
    - Ensure the items are sorted in descending order of models
3. When handling known manufacturer's warranty exceptions: 
    - 
    - Ensure the right exception is raised with the appropriate messages and reason.
4. When handling vehicle warranty without exception:
    - 
    - Ensure the function returns the exception flag and a single item for the correct warranty
5. When no vehicle warranty items are found:
    - Ensure the `VehicleUnavailable` exception is raised correctly with the appropriate message and vehicle information.


In [7]:
print(response['unit_test_script'])

# import necessary packages and modules
import unittest
from unittest.mock import MagicMock, Mock
from glue_code import (get_warranty_details_with_exception_check, 
                       NotEligibleForFCheck, 
                       VehicleUnavailable)

# define a class of test cases to test our function
class TestGetWarrantyDetailsWithExceptionCheck(unittest.TestCase):
    
    def setUp(self):
        # set up mock CosmosDB client
        self.cosmos_db = Mock()
        self.container_client = Mock()
        self.container_client.query_items.return_value = []
        self.cosmos_db.get_container_client.return_value = self.container_client

    def test_connecting_to_cosmos_db(self):
        # Test that an exception is raised if cosmos_db is None
        with self.assertRaises(Exception) as cm:
            get_warranty_details_with_exception_check('Toyota', None, 2022, None)
        self.assertEqual(str(cm.exception), str("cosmos_db should not be None"))

        # Test that an excep

In [178]:
code = '''
using System.ComponentModel.DataAnnotations;
using System.Text.RegularExpressions;
using MoZuesOutboundService.Business.Helper;

namespace MoZuesOutboundService.Business.Model.CashBash;


public record CashBashAttendeeModel : CashBashBaseModel
{
    [Required] public AttendeeDataModel Attendee { get; set; } = new();
}

public record AttendeeDataModel
{
    public string JmaAttendeeId { get; set; } = string.Empty;
    public string VendorId { get; set; } = string.Empty;
    private string _mobile = string.Empty;

    [RegularExpression(@"^\d{10}", ErrorMessage = "Invalid phone number")]
    [Required]
    public string MobilePhoneNumber
    {
        get
        {
            if (!string.IsNullOrEmpty(_mobile))
            {
                var phone = Regex.Replace(_mobile, @"[^\d]", "");
                return phone.Length > 10 ? phone[..10] : phone;
            }
            return _mobile;
        }
        set => _mobile = value;
    } 

    [EmailAddress][Required]  public string EmailAddress { get; set; } = string.Empty;
    [Required] public string FirstName { get; set; } = string.Empty;
    [Required] public string LastName { get; set; } = string.Empty;
    public string Title { get; set; } = string.Empty;
    [Required] public string Address { get; set; } = string.Empty;
    [Required] public string City { get; set; } = string.Empty;
    private string _state = string.Empty;

    [StringLength(2, MinimumLength = 2, ErrorMessage = "State must be 2 characters")]
    [Required]
    public string State
    {
        get
        {
            if (!string.IsNullOrEmpty(_state) && _state.Length > 2)
            {
                return _state[..2];
            }

            return _state;
        }
        set => _state = value;
    }

    private string _zip = string.Empty;

    [Required(AllowEmptyStrings = false)]
    [RegularExpression(@"^\d{5}?$", ErrorMessage = "Invalid Zip")]
    public string Zip
    {
        get
        {
            if (!string.IsNullOrEmpty(_zip) && _zip.Length > 5)
            {
                return _zip[..5];
            }

            return _zip;
        }
        set => _zip = value;
    }

    [Required] [StringRange(AllowableValues = ["Absentee", "Attendee","Guest", "Dealer","Staff", "Vendor","Host", "Prospect"])]
    public string Type { get; set; } = string.Empty;
    [Required] public string CventApprovalStatus { get; set; } = string.Empty;
    public string GuestRecordId { get; set; } = string.Empty;
    [Required] public string ContestId { get; set; } = string.Empty;
}

using Microsoft.Extensions.Logging;
using MoZuesOutboundService.Business.Helper;
using MoZuesOutboundService.Business.Model;
using MoZuesOutboundService.Business.Model.CashBash;
using MoZuesOutboundService.Business.Model.Cvent;
using MoZuesOutboundService.Business.Model.SalesConnect;
using MoZuesOutboundService.Business.Repository;
using Metadata = MoZuesOutboundService.Business.Model.CashBash.Metadata;
using ValidationResults = MoZuesOutboundService.Business.Model.CashBash.ValidationResults;

namespace MoZuesOutboundService.Business.Services;

'''

In [172]:
service_code = '''public class CashBashAttendeeService : ICashBashAttendeeService
{
    private readonly ILogger<CashBashAttendeeService> _logger;
    private readonly ICashBashAttendeeRepository _cashBashAttendeeRepository;
    private readonly ICashBashPullTeamRepository _cashBashPullTeamRepository;
    private readonly IOutBoundHttpService _outBoundHttpService;


    public CashBashAttendeeService(ILogger<CashBashAttendeeService> logger
        , ICashBashAttendeeRepository cashBashAttendeeRepository
        ,ICashBashPullTeamRepository cashBashPullTeamRepository
        , IOutBoundHttpService outBoundHttpService
        )
    {
        _logger = logger;
        _cashBashAttendeeRepository = cashBashAttendeeRepository;
        _cashBashPullTeamRepository = cashBashPullTeamRepository;
        _outBoundHttpService = outBoundHttpService;
    }

    public async Task<CashBashAttendeeModel> UpdateFromScAttendee(SalesConnectAttendeeModel scSalesConnectAttendee)
    {
        var targetAttendee = await _cashBashAttendeeRepository.GetCashBashByAttendeeId(scSalesConnectAttendee.Id);
        if (targetAttendee is not null)
        {//Records exist
            _logger.LogInformation("Updating existing attendee {Id}", scSalesConnectAttendee.Id);

            //if cvent data present then partial merge if registered
            var cventData = await _cashBashAttendeeRepository.GetCventByJmaAttendeeId(scSalesConnectAttendee.Id);
            if (cventData is not null)
            {
                await UpdatePartialFromSalesConnectCashBashAttendeeModel(scSalesConnectAttendee, targetAttendee);
            }
            else
            {
                await UpdateFullFromSalesConnectCashBashAttendeeModel(scSalesConnectAttendee, targetAttendee);
            }
        }
        else
        { //New Record
            _logger.LogInformation("Creating new attendee {Id}", scSalesConnectAttendee.Id);

            targetAttendee = new CashBashAttendeeModel
            {
                Id = Guid.NewGuid().ToString(),
                Metadata = new Metadata
                {
                    DestinationCreatedTimeStamp =  DateTime.UtcNow
                }
            };
            await UpdateFullFromSalesConnectCashBashAttendeeModel(scSalesConnectAttendee, targetAttendee);
        }

        await ValidateCashBashAttendee(targetAttendee);
        
        return targetAttendee;
    }

    public async Task<CashBashAttendeeModel> UpdateFromCventAttendee(CventContact cventContact)
    {
        
        var targetAttendee = await _cashBashAttendeeRepository.GetCashBashByAttendeeId(cventContact.Contact.JmaAttendeeId);
        if (targetAttendee is not null)
        {
            //Attendee Records exist
            await UpdateFromCvent(cventContact, targetAttendee);
            targetAttendee.Attendee.GuestRecordId = await UpdateGuest(cventContact, targetAttendee.Attendee.GuestRecordId);
        }
        else
        {
            //Cvent non Attendee
            targetAttendee = await _cashBashAttendeeRepository.GetCashBashByRecordId(cventContact.Id);
            
            if (targetAttendee is not null)
            {//Existing non-Attendee

                await UpdateFromCvent(cventContact, targetAttendee);
            }
            else
            {//New Record
                targetAttendee = new CashBashAttendeeModel
                {
                    Id = string.IsNullOrEmpty(cventContact.Contact.JmaAttendeeId)
                        ? cventContact.Id
                        : cventContact.Contact.JmaAttendeeId,
                    Metadata = new Metadata
                    {
                        SourceCreatedTimeStamp = DateTime.UtcNow,
                        DestinationCreatedTimeStamp = DateTime.UtcNow
                    }
                };
                await UpdateFromCvent(cventContact, targetAttendee);
                targetAttendee.Attendee.GuestRecordId = await CreateGuestRecord(cventContact);
            }
        }
        
        await ValidateCashBashAttendee(targetAttendee);
        //Update data
        await StoreAttendeeRecord(targetAttendee);
        return targetAttendee;
    }
    public async Task<bool> SendAttendeeHttp(CashBashAttendeeModel cbAttendee)
    {
        cbAttendee.MoZusApiResponse = await _outBoundHttpService.SendAttendee(cbAttendee);
        cbAttendee.Metadata.PreviousUpdatedByEvent = cbAttendee.Metadata.LastUpdateByEvent;
        cbAttendee.Metadata.LastUpdateByEvent = LastUpdatedByEnum.MoZuesApiService;

        var id = await StoreAttendeeRecord(cbAttendee);
        return !string.IsNullOrEmpty(id) && cbAttendee.MoZusApiResponse.HttpStatusCode == 200;
    }

    private async Task<bool> IsDuplicateCashBashEmail(string email, string currentAttendeeId, string contestId)
    {
        var records = await _cashBashAttendeeRepository.GetCashBashByEmail(email, contestId);
        return records is not null
               && records.Any(a => a.Id != currentAttendeeId);
    }

    public async Task<bool> ValidateCashBashAttendee(CashBashAttendeeModel cbAttendee)
    {
        cbAttendee.Metadata.IsComplete = true;

        var validationIssues = new List<ValidationResults>();
        var items= ModelValidationHelper.ValidateModel(cbAttendee);
        
        if (items.Any())
        {
            cbAttendee.Metadata.IsComplete = false;
            validationIssues.AddRange(items
                .Select(s =>
                    new ValidationResults { Message = $"{s.ErrorMessage}" }));
        }
       
        //check for cvent Data
        if (cbAttendee.Attendee.CventApprovalStatus != "Confirmed")
        {
            validationIssues.Add(new ValidationResults { Message = "Cvent Registration Incomplete" });
        }    
        
        //check unique email address
        if (!string.IsNullOrEmpty(cbAttendee.Attendee.ContestId))
        {
            var isDuplicateEmail = await IsDuplicateCashBashEmail(cbAttendee.Attendee.EmailAddress
                , cbAttendee.Id
                , cbAttendee.Attendee.ContestId);
            if (isDuplicateEmail)
            {
                validationIssues.Add(new ValidationResults { Message = "Duplicate E-mail" });
            }
        }

        if (validationIssues.Any())
        {
            cbAttendee.Metadata.ValidationResults = validationIssues;
            cbAttendee.Metadata.IsComplete = false;
        }
        else
        {
            cbAttendee.Metadata.ValidationResults = [];
            cbAttendee.Metadata.IsComplete = true;
        }
        //Update data
        await StoreAttendeeRecord(cbAttendee);
        return cbAttendee.Metadata.IsComplete;
    }
    private async Task<string> UpdateGuest(CventContact cventContact, string attendeeGuestRecordId)
    {
        var guestRecord = await _cashBashAttendeeRepository.GetCashBashByRecordId(attendeeGuestRecordId);
        if (guestRecord is null)
        {
            //Create new Guest
            return await CreateGuestRecord(cventContact);
        }

        //Update Guest
        guestRecord.Attendee.FirstName = cventContact.Contact.GuestFirstName;
        guestRecord.Attendee.LastName = cventContact.Contact.GuestLastName;
        guestRecord.Attendee.EmailAddress = cventContact.Contact.GuestEmail;
        guestRecord.Attendee.MobilePhoneNumber = string.IsNullOrEmpty(cventContact.Contact.GuestPhoneNumber) ? 
            cventContact.Contact.MobileNumber :cventContact.Contact.GuestPhoneNumber;
        guestRecord.Attendee.CventApprovalStatus = cventContact.Contact.RegistrationStatus;
        //guestRecord.Metadata.IsComplete = cventContact.Contact.RegistrationStatus == "Confirmed";

        guestRecord.Attendee.Address = cventContact.Contact.Address;
        guestRecord.Attendee.City = cventContact.Contact.City;
        guestRecord.Attendee.State = cventContact.Contact.State;
        guestRecord.Attendee.Zip = cventContact.Contact.Zip;

        if (string.IsNullOrEmpty(guestRecord.Attendee.ContestId))
            guestRecord.Attendee.ContestId = await GetContestId(cventContact.Contact.JmaAttendeeId);

        //Update Metadata
        guestRecord.Metadata.DocumentUpdateTimeStamp = DateTime.UtcNow;

        guestRecord.Metadata.LastUpdateByEvent = LastUpdatedByEnum.CventService;
        guestRecord.Metadata.DestinationUpdatedTimeStamp =
            guestRecord.Metadata.SourceUpdatedTimeStamp = DateTime.UtcNow;
       
        guestRecord.Metadata.CheckSum = CheckSumHelper.GetCheckSum(guestRecord, ["Attendee"]);
        await ValidateCashBashAttendee(guestRecord);
        await StoreAttendeeRecord(guestRecord);
        return guestRecord.Id;
    }
    private async Task<string> GetContestId(string attendeeId)
    {
        _logger.LogInformation("Getting contest ID for attendee {id}", attendeeId);
        return await _cashBashPullTeamRepository.GetContestIdByAttendee(attendeeId);
    }
    private async Task<string> CreateGuestRecord(CventContact cventContact)
    {
        if (cventContact.Contact.RegistrationType == "Attendee")
        {
            if (
                !string.IsNullOrEmpty(cventContact.Contact.GuestEmail) &&
                !string.IsNullOrEmpty(cventContact.Contact.GuestFirstName) &&
                !string.IsNullOrEmpty(cventContact.Contact.GuestLastName)
            )
            {
                var guestRecord = new CashBashAttendeeModel
                {
                    Metadata = new Metadata
                    {
                        DestinationCreatedTimeStamp =  DateTime.UtcNow,
                        DocumentUpdateTimeStamp = DateTime.UtcNow,
                        LastUpdateByEvent = LastUpdatedByEnum.CventService,
                        SourceUpdatedTimeStamp = DateTime.UtcNow,
                        SourceCreatedTimeStamp = DateTime.UtcNow,
                        DeleteStatus = "N",
                        IsComplete = cventContact.Contact.RegistrationStatus == "Confirmed"
                    },
                    Id = Guid.NewGuid().ToString(),
                    Attendee = new AttendeeDataModel
                    {
                        FirstName  = cventContact.Contact.GuestFirstName,
                        LastName = cventContact.Contact.GuestLastName,
                        EmailAddress = cventContact.Contact.GuestEmail,
                        MobilePhoneNumber = string.IsNullOrEmpty(cventContact.Contact.GuestPhoneNumber) ? 
                            cventContact.Contact.MobileNumber :cventContact.Contact.GuestPhoneNumber,
                        Type = "Guest",
                        Address = cventContact.Contact.Address,
                        City = cventContact.Contact.City,
                        State = cventContact.Contact.State,
                        Zip = cventContact.Contact.Zip,
                        CventApprovalStatus = cventContact.Contact.RegistrationStatus
                    },
                };
                if (string.IsNullOrEmpty(guestRecord.Attendee.ContestId))
                {
                    guestRecord.Attendee.ContestId = await GetContestId(cventContact.Contact.JmaAttendeeId);
                }
                guestRecord.Metadata.CheckSum = CheckSumHelper.GetCheckSum(guestRecord, ["Attendee"]);
                await ValidateCashBashAttendee(guestRecord);
                return await StoreAttendeeRecord(guestRecord);
            }
        }

        return string.Empty;
    }
    private async Task UpdateFullFromSalesConnectCashBashAttendeeModel(
        SalesConnectAttendeeModel scSource, CashBashAttendeeModel cbTarget)
    {
        cbTarget.Metadata.DeleteStatus = scSource.Attendee.Metadata.DeleteStatus;
        cbTarget.Metadata.DeleteTimeStamp = scSource.Attendee.Metadata.DeleteTimeStamp;
        cbTarget.Metadata.DocumentUpdateTimeStamp = DateTime.UtcNow;
        cbTarget.Metadata.SourceCreatedTimeStamp = scSource.Attendee.Metadata.SourceCreatedTimeStamp;
        cbTarget.Metadata.SourceUpdatedTimeStamp = scSource.Attendee.Metadata.SourceUpdatedTimeStamp;
        cbTarget.Metadata.LastUpdateByEvent = LastUpdatedByEnum.SalesConnectService;
        cbTarget.Metadata.DestinationUpdatedTimeStamp = DateTime.UtcNow;

        cbTarget.Attendee.JmaAttendeeId = scSource.Id;
        cbTarget.Attendee.Address = scSource.Attendee.Address;
        cbTarget.Attendee.City = scSource.Attendee.City;
        cbTarget.Attendee.State = scSource.Attendee.State;
        cbTarget.Attendee.Zip = scSource.Attendee.Zip;
        cbTarget.Attendee.Title = scSource.Attendee.Title;
        cbTarget.Attendee.Type = scSource.Attendee.Type == "Absentee" ? "Absentee" : "Attendee";
        cbTarget.Attendee.EmailAddress = scSource.Attendee.EmailAddress;
        cbTarget.Attendee.FirstName = scSource.Attendee.FirstName;
        cbTarget.Attendee.LastName = scSource.Attendee.LastName;
        cbTarget.Attendee.MobilePhoneNumber = scSource.Attendee.MobilePhoneNumber;
        if (string.IsNullOrEmpty(cbTarget.Attendee.ContestId))
        {
            cbTarget.Attendee.ContestId = await GetContestId(cbTarget.Attendee.JmaAttendeeId);
        }

        cbTarget.Metadata.CheckSum = CheckSumHelper.GetCheckSum(cbTarget, ["Attendee"]);
    }
    private async Task UpdatePartialFromSalesConnectCashBashAttendeeModel(SalesConnectAttendeeModel scSource, CashBashAttendeeModel cbTarget)
    {
        cbTarget.Metadata.DeleteStatus = scSource.Attendee.Metadata.DeleteStatus;
        cbTarget.Metadata.DeleteTimeStamp = scSource.Attendee.Metadata.DeleteTimeStamp;
        cbTarget.Metadata.DocumentUpdateTimeStamp = DateTime.UtcNow;
        cbTarget.Metadata.SourceCreatedTimeStamp = scSource.Attendee.Metadata.SourceCreatedTimeStamp;
        cbTarget.Metadata.SourceUpdatedTimeStamp = scSource.Attendee.Metadata.SourceUpdatedTimeStamp;
        cbTarget.Metadata.LastUpdateByEvent = LastUpdatedByEnum.SalesConnectService;
        cbTarget.Metadata.DestinationUpdatedTimeStamp = DateTime.UtcNow;
        
        cbTarget.Attendee.Type = scSource.Attendee.Type == "Absentee" ? "Absentee" : "Attendee";
        
        if (string.IsNullOrEmpty(cbTarget.Attendee.ContestId))
        {
            cbTarget.Attendee.ContestId = await GetContestId(cbTarget.Attendee.JmaAttendeeId);
        }
        
        cbTarget.Metadata.CheckSum = CheckSumHelper.GetCheckSum(cbTarget, ["Attendee"]);
    }
    private async Task UpdateFromCvent(CventContact cventSource, CashBashAttendeeModel cbTarget)
    {
        cbTarget.Metadata.DocumentUpdateTimeStamp = DateTime.UtcNow;
       
        cbTarget.Metadata.LastUpdateByEvent = LastUpdatedByEnum.CventService;
        cbTarget.Metadata.DestinationUpdatedTimeStamp = 
        cbTarget.Metadata.SourceUpdatedTimeStamp = DateTime.UtcNow;
        
        cbTarget.Attendee.JmaAttendeeId = cventSource.Contact.JmaAttendeeId;
        cbTarget.Attendee.VendorId = cventSource.Contact.VendorId;
        cbTarget.Attendee.Address = cventSource.Contact.Address;
        cbTarget.Attendee.City = cventSource.Contact.City;
        cbTarget.Attendee.State = cventSource.Contact.State;
        cbTarget.Attendee.Zip = cventSource.Contact.Zip;
       
        cbTarget.Attendee.Title = cventSource.Contact.Title;
        cbTarget.Attendee.Type = cventSource.Contact.RegistrationType;
        cbTarget.Attendee.EmailAddress = cventSource.Contact.EmailAddress;
        cbTarget.Attendee.FirstName = cventSource.Contact.FirstName;
        cbTarget.Attendee.LastName = cventSource.Contact.LastName;
        cbTarget.Attendee.MobilePhoneNumber = cventSource.Contact.MobileNumber;
        cbTarget.Attendee.CventApprovalStatus = cventSource.Contact.RegistrationStatus;
        
        if (string.IsNullOrEmpty(cbTarget.Attendee.ContestId))
        {
            cbTarget.Attendee.ContestId = await GetContestId(cbTarget.Attendee.JmaAttendeeId);
        }
        cbTarget.Metadata.CheckSum = CheckSumHelper.GetCheckSum(cbTarget, ["Attendee"]);

    }
    private async Task<string> StoreAttendeeRecord(CashBashAttendeeModel cbAttendee)
    {
        _logger.LogInformation("Updating CashBash {id} attendee to cosmos", cbAttendee.Id);
        return await _cashBashAttendeeRepository.UpsertAttendee(cbAttendee);
    }
}'''

In [185]:
print(code)


using System.ComponentModel.DataAnnotations;
using System.Text.RegularExpressions;
using MoZuesOutboundService.Business.Helper;

namespace MoZuesOutboundService.Business.Model.CashBash;


public record CashBashAttendeeModel : CashBashBaseModel
{
    [Required] public AttendeeDataModel Attendee { get; set; } = new();
}

public record AttendeeDataModel
{
    public string JmaAttendeeId { get; set; } = string.Empty;
    public string VendorId { get; set; } = string.Empty;
    private string _mobile = string.Empty;

    [RegularExpression(@"^\d{10}", ErrorMessage = "Invalid phone number")]
    [Required]
    public string MobilePhoneNumber
    {
        get
        {
            if (!string.IsNullOrEmpty(_mobile))
            {
                var phone = Regex.Replace(_mobile, @"[^\d]", "");
                return phone.Length > 10 ? phone[..10] : phone;
            }
            return _mobile;
        }
        set => _mobile = value;
    } 

    [EmailAddress][Required]  public string

In [182]:
requirements = '''In order to process attendee data received from Event Hub,
the Azure Functions App developer
needs to create appropriate attendee processing logic.

Given that Azure DigiHub sends attendee data to Azure Functions App,
When data is received in Azure Functions,
Then appropriate attendee processing logic should be executed based on the received data
'''
unit_test_framework = "Xunit"

response = test_endpoint(code, requirements, unit_test_framework)

In [183]:
print(response['unit_test_script'])

```C#
# packages
using System;
using Xunit;
using Moq;

# use/import Xunit for our unit tests


# unit tests with Xunit
public class AttendeeDataModelTests
{
    [Fact]
    public void MobilePhoneNumber_ShouldReturnTruncatedPhoneNumber_WhenMoreThan10DigitsAreEntered()
    {
        // Arrange
        var attendee = new AttendeeDataModel();
        attendee.MobilePhoneNumber = "12345678900";

        // Act
        var result = attendee.MobilePhoneNumber;

        // Assert
        Assert.Equal("1234567890", result);
    }
	
    [Fact]
    public void MobilePhoneNumber_ShouldReturnOnlyDigits_WhenSpecialCharactersAreEntered()
    {
        // Arrange
        var attendee = new AttendeeDataModel();
        attendee.MobilePhoneNumber = "123-456-7890";

        // Act
        var result = attendee.MobilePhoneNumber;

        // Assert
        Assert.Equal("1234567890", result);
    }

    [Fact]
    public void State_ShouldReturnTruncatedState_WhenMoreThan2CharacterStateIsEntered()
    {
  

In [11]:
print(response['test_plan'])

Here are some diverse scenarios that the code should handle:

1. Validating attendees' data:
    - Attending a different event in the same location - should not be allowed.
    - Attending multiple sessions on the same day - should not be allowed.
    - Registering two attendees with the same email address - should not be allowed.
2. Updating an attendee's data based on other API services:
    - SalesConnect attendee data provided by operations manager.
    - Cvent attendee data provided by the event registration service team.
    - Raising an exception when the provided data has null, or an empty value is provided.
3. Storing and retrieving attendee data in Cosmos DB.
4. Handling absentees or guests:
    - No Guest details are provided.
    - A guest has completed the registration.
    - Guest is not married to the attending attendee.
    - Multiple guest records for the same attendee.
5. Updating attendee data after registration:
    - Updating an attendee's details after registratio

In [14]:
# print(response['unit_test_script'])
response['unit_test_script']

'# packages\nusing System;\nusing System.Collections.Generic;\nusing System.Threading.Tasks;\nusing Microsoft.Extensions.Logging;\nusing Moq;\nusing Xunit;\n\n# imports\nusing CashBash.Core.Interfaces;\nusing CashBash.Core.Models;\nusing CashBash.Core.Services;\n\npublic class CashBashAttendeeServiceTests\n{\n    private readonly Mock<ILogger<CashBashAttendeeService>> _loggerMock = new();\n    private readonly Mock<ICashBashAttendeeRepository> _cashBashAttendeeRepositoryMock = new();\n    private readonly Mock<ICashBashPullTeamRepository> _cashBashPullTeamRepositoryMock = new();\n    private readonly Mock<IOutBoundHttpService> _outBoundHttpServiceMock = new();\n    private readonly CashBashAttendeeService _sut;\n\n    public CashBashAttendeeServiceTests()\n    {\n        _sut = new CashBashAttendeeService(_loggerMock.Object,\n            _cashBashAttendeeRepositoryMock.Object,\n            _cashBashPullTeamRepositoryMock.Object,\n            _outBoundHttpServiceMock.Object);\n    }\n\n

In [2]:
import requests
import json
from dotenv import load_dotenv

load_dotenv('testing.env')

True

In [3]:
import os
import tiktoken
from openai import AzureOpenAI
from azure.identity import ClientSecretCredential
from util.text_processing import *

def call_chat(self, messages, seed = None, temperature = 1):
    response = self.chat.completions.create(
                model=os.environ['openai_deployment_name'],
                messages=messages,
                seed=seed
                )
    message_response = response.choices[0].message.content
    return message_response

def get_openai_client(api_version = "2023-12-01-preview"):

    credential = ClientSecretCredential(tenant_id=os.getenv('openai_tenant_id'), 
                                        client_id=os.getenv('openai_client_id'), 
                                        client_secret=os.getenv('openai_client_secret'))
    
    jwt = credential.get_token("api://{}".format(os.getenv('openai_credential_scope')))
    token = jwt.token
    AzureOpenAI.call_chat = call_chat
    client = AzureOpenAI(
        api_key = token,  
        api_version=api_version,
        azure_endpoint = os.getenv('openai_api_base'),
        )
    return client

In [177]:
client = get_openai_client('2023-12-01-preview')

In [ ]:
req = '''As a Claims Processing System,
I want to implement exception handling for new vehicles with a model year between 2007 and 2009,
So that the correct warranty terms are applied based on the sale date, enhancing claim processing efficiency.

Exception Rule:
The 2009 - 2007 standard lifetime powertrain warranty started on July 26, 2007, and was included with vehicles sold on or after this date until the last 2009 model was sold.
Scope:
New Vehicles: In-Service Miles ≤ 500 miles
Make: Jeep
MY Start: 2007
MY End: 2009
Notes:
This user story focuses on implementing exception handling for vehicles whose model year is between 2007 and 2009, ensuring accurate application of warranty terms.
The acceptance criteria detail the specific conditions under which the system should apply each type of warranty (standard or lifetime Powertrain) based on the sale date.
Exception handling enhances claim processing efficiency by ensuring appropriate warranty coverage for vehicles within the specified timeframe.'''

In [192]:
list_sys = '''
You are an expert business analyst who is highly experienced in behavioral driven development and developing well-constructed acceptance criteria from technical requirements.
When supplied an enabler story description, create rule-oriented acceptance criteria.

Rule-oriented acceptance criteria is typically used when defining a set of standards which the system should have in order to consider the user story complete.  It's typically defined in numbered lists or bullets. 

1. Output as one entire snippet for easy copying. Don't add a heading like "Acceptance Criteria".
2. Provide positive and negative scenarios.
3. Do not make assumptions about variables or data that the business analyst does not provide. Instead of making up false details, keep the rules relatively generic if the business analyst doesn't provide lots of detail.
'''

ba_enabler_story = '''
{enabler_type} Enabler Story:
{story}
'''

In [200]:
req_text = '''As a BI Engineer
I want to evaluate AD Group Access for Certified Datasets Read Only (Backoffice) for Field User Implications
So that I can confirm that providing access to this AD group for field users will not open all access (therefore bypass RLS rules)

*Note: Confirm with Sales Ops how it works for BackOffice role vs the specific sales roles (ZVP, DDP, DPM)
-- Backoffice role gives Sales Tracker view for all Zones, Districts, Dealers 
Problem Statement: IF we use this AD group for all other Certified Datasets, will it allow too much access for Field Users on the Sales Tracker'''


response = client.call_chat(messages = [{"role": "system", "content": list_sys}, 
                        {"role": "user","content": ba_enabler_story.format(story=req_text, enabler_type='')}
                    ], seed = None)
print(response)

1. The AD group access for Certified Datasets Read Only (Backoffice) should only provide access to the Sales Tracker view for all Zones, Districts, and Dealers.
2. Confirm with Sales Ops the different Backoffice role compared to the specific sales roles (ZVP, DDP, DPM) and ensure that the AD group access does not provide access to any other sales roles.
3. Verify that access is limited to only read-only permissions.
4. Ensure that the field users, who have access to the AD group for Certified Datasets Read Only (Backoffice), should not be able to bypass RLS rules.
5. Negative Scenario: The AD group access should not provide any access to modify, delete or insert the data.
6. Negative Scenario: The AD group access should not include any other dataset besides the Certified Datasets.
7. Negative Scenario: The AD group access should not provide any access to any confidential data that is not relevant to the Backoffice role and Sales Tracker view.


In [ ]:
'''For a story to be considered complete, they should follow the principles of INVEST:

Our goal is to provide the user suggestions on any of the above points if the AI finds that the BA's story is lacking. 

Independent – can be implemented in any order on its own​
Negotiable –"what" and "why" are defined, but "how" is not outlined
Value – has business value on its own​
Estimable – can be estimated the way it stands​
Small – can be implemented in 1-3 days​ in order to limit stories in progress
Testable – is clear enough to write good acceptance criteria and test cases from it​, should be possible to test'''

In [132]:
user_story_val_sys = '''
You are a world-class business analyst who helps other business analysts by evaluating the content of user story descriptions.
Note that you will not be evaluating any acceptance criteria- you will be given the description of the user story and you are expected to evaluate only that.

As you know, user story descriptions must meet the following criteria:
* Explain why the work is valuable to the end user
* Describe what the requirements are in detail
* What and why, not the how: Do not describe how the work should be implemented
* Be specific and precise in defining what is required
* Use clear plain language

To evaluate the story description, score the provided user story description to evaluate if the story description meets the criteria.
Create lists to explain your suggestions and feedback.

Scoring Scale of 0 to 5:
0 does not meet the criteria.
5 meets all the criteria.

Your entire response should be in this JSON format, as follows:
{
    "Overall Score" : The overall score goes here, just the number value.
    "Meets Criteria" : [] Here, list the aspects of the story description that meet the criteria above. This can be blank. ex. Consider...
    "Suggestions" : [] Here, list specific suggestions to the business analyst on how to improve the user story description based on the criteria above. This can be blank. ex. Consider...
}
'''

user_story_val_user = '''
{story}
'''

In [121]:
# user_story_val_sys = '''
# Act as a business analyst to validate that the requested user story meets the following requirements.

# * A user story is defined as an informal explanation of a software feature written from the perspective of an end user.
# * The purpose of a story is to articulate piece of work and why it is important.
# * A user story should express what needs to be done in plain language.
# * A user story should NEVER express the "how", only the "what" and "why".
# * The story should NOT describe the approach to take, how to complete the requirement, or how the work will be done.

# User stories should follow the principles of INVEST:
# Independent: can be implemented in any order on its own
# Negotiable: "what" and "why" are defined, but "how" is NOT outlined. No implementation details and no "how".
# Value: has business value on its own
# Estimable: is detailed enough to be estimated the way it stands

# To evaluate the story, score the user story to evaluate if the story meets the requirements and INVEST principles.
# Create lists that contains your explanations.

# Scoring Scale of 0 to 5:
# 0, does not meet requirements or INVEST principles.
# 5, meets the requirements and follows the INVEST principles.

# Your entire response should be in this JSON format, as follows:
# {
#     "Overall Score" : The overall score goes here, just the number value.
#     "Meets Requirements" : [] Here, list the aspects of the story that meet the requirements above. This can be blank. ex. Consider...
#     "Suggestions" : [] Here, list specific suggestions to the author on how to improve the user story based on the requirements above. This can be blank. ex. Consider...
# }
# '''


# user_story_val_user = '''
# Validate the following user story:
# {story}
# '''


In [166]:
req_text = '''as a Compliance Specialist

I want to View all products associated with a VIN

so that to understand coverages and provide support'''

# req_text = '''As a Service Advisor,
# I want to know if a Claim has been previously submitted prior to submitting a Claim,
# So that I can easily identify and prevent duplicate claims.

# NOTE: 
# Salesforce: "Is submission Unique based on the criteria identified?" 
#     Vehicle VIN, Service Contract, & Repair Order number
# .NET/API: "Is Duplicate?"
#     "Yes/No"
# Includes UI/UX
#     Tentative mockup:
#     'Duplicate Claim Detected'
#     'A similar claim with the same Vehicle VIN, Contract Number, and Repair Order Number was submitted on [Month Day, Year]. Please review your submission details to ensure they are correct.
#     Vehicle VIN: [Vehicle VIN Number]
#     Contract: [Contract Number]
#     Repair Order: [Repair Order Number]
#     If you believe this is an error, contact support for further assistance.' '''

# req_text = '''As a BI Engineer
# I want to evaluate AD Group Access for Certified Datasets Read Only (Backoffice) for Field User Implications
# So that I can confirm that providing access to this AD group for field users will not open all access (therefore bypass RLS rules)

# *Note: Confirm with Sales Ops how it works for BackOffice role vs the specific sales roles (ZVP, DDP, DPM)
# -- Backoffice role gives Sales Tracker view for all Zones, Districts, Dealers 
# Problem Statement: IF we use this AD group for all other Certified Datasets, will it allow too much access for Field Users on the Sales Tracker'''

# req_text = '''As a QA Tester,
# I want to validate the accuracy and functionality of the bulk ingested Forms,
# So that we can ensure the AI Service correctly processes claims based on the coverage and terms defined in these forms.

# Set of Forms to Ingest:

# KPLW-0521	Not Yet Ingested	282	5.83%	Certified pre-owned
# KPLW-0118	Not Yet Ingested	247	5.10%	Certified pre-owned
# KPLW-0115	Not Yet Ingested	45	0.93%	Certified pre-owned
# KC11-0321	Not Yet Ingested	23	0.48%	Certified pre-owned
# VL11-0117	Not Yet Ingested	11	0.23%	Certified pre-owned
# KC11-0115	Not Yet Ingested	7	0.14%	Certified pre-owned
# KC11-1020	Not Yet Ingested	7	0.14%	Certified pre-owned
# CK11-1020	Not Yet Ingested	6	0.12%	Certified pre-owned
# KC11-1118	Not Yet Ingested	5	0.10%	Certified pre-owned
# ASKC-0321	Not Yet Ingested	3	0.06%	Certified pre-owned
# KC11-1119	Not Yet Ingested	3	0.06%	Certified pre-owned
# '''

# req_text = '''As a Claims Admin,
# I want delete the Factory New and Factory Used toggle
# And have one single toggle for Factory
# So that I the user is not confused by the available toggles and their functionality.
# '''

# req_text = '''As a VFI Specialist/Manager
# I want the case status to automatically update to 'reporting' once Zoom meeting is ended
# So that the deal tracker button is visible in a timely manner for me to complete the deal

# Once VFI specialist ends the zoom meeting, the case status is supposed to go from 'in progress' to 'reporting'. This automation is not working properly and VFI specialists have to manually refresh the case tab (or click somewhere in the case) for the case to go to reporting status in order for the Deal Tracker button to appear. The Zoom start and end time are populating in a timely manner, but the case status is not going to reporting timely. 

# Copied from Repro Steps when changed from Bug to Story:
# Submit deal in Dealer Source
# Accept deal in SalesConnect
# Create Zoom link 
# Open Zoom link from case (note case goes to in progress status)
# End zoom link 
#     at this point, the start and end time of zoom populate in the case after about 20 seconds, but the case status does not change to reporting automatically. It will change if tab is refreshed or if something else is clicked on the case. '''

In [84]:
# Small: can be implemented in 1-3 days in order to limit stories in progress
# Testable: is clear enough to write good acceptance criteria and test cases, story is possible to test

In [76]:
# "None", "No suggestions"

In [169]:
response = client.call_chat(messages = [{"role": "system", "content": user_story_val_sys}, 
                        {"role": "user","content": user_story_val_user.format(story=req_text)}
                    ], seed = None)
print(response)

{
    "Overall Score" : 3,
    "Meets Criteria" : [
        "Explains why the work is valuable to the end user",
        "Describes what the requirements are in detail",
        "Uses clear plain language"
    ],
    "Suggestions" : [
        "Be more specific in defining what is required",
        "Avoid using vague terms such as 'understand coverages'",
        "Avoid using jargon specific to the company or position ('Compliance Specialist', 'VIN')"
    ]
}


In [41]:
p = '''
Validate if the given story contains a strong or a weak persona. Examples of weak personas are: user, buyer, customer, representative. Examples of strong personas are: prospective car buyer, registered member, member sales representative, customer website admin.
If the story contains a strong persona, respond only with "Strong."
If the story contains a weak persona, respond with a brief suggestion like "Consider making the persona stronger or more specific so the persona's motivation is clearer."
'''

p = '''
Validate if the given story contains a strong or a weak persona. Examples of weak personas are: user, buyer, customer, representative. Examples of strong personas are: prospective car buyer, registered member, member sales representative, customer website administrator.
If the story contains a strong persona, respond only with "Strong."
If the story contains a weak persona, respond only with "Weak."
'''

req_text = '''As a user, I need item velocity code rules added for p-coded parts So that the P-Coded parts are not removed from stocking during item eligibility updates in the P&F system.'''
# parts analyst
response = client.call_chat(messages = [ {"role": "system","content": p},
                    {"role": "user","content": req_text}
                ], seed = None)

if response.startswith('Weak.'):
    response = client.call_chat(messages = [ {"role": "system","content": p},
                    {"role": "user","content": req_text},
                    {"role" : "assistant" , "content" : response},
                    {"role" : "user", "content" : "Respond with a very brief, one sentence suggestion of how and why to improve the persona. Include the existing persona somewhere in your response."}
                ], seed = None)
response

'Consider improving the persona "user" by adding more context to the user\'s role or goal, such as "maintenance crew user" or "budget-conscious user," for better decision making and prioritization in product development.'

In [41]:
p = '''
Validate if the given text is written in this format: "In order to (do something) the (technical user) needs to/must (do a technical thing)".
Respond with only "Yes." or "No.".
'''

In [42]:

req_text = '''As a parts analyst, I need item velocity code rules added for p-coded parts So that the P-Coded parts are not removed from stocking during item eligibility updates in the P&F system.'''

response = client.call_chat(messages = [ {"role": "system","content": p},
                    {"role": "user","content": req_text}
                ], seed = None)

response

'No.'

In [93]:
p = '''
You are a skilled agile coach. You support business analysts writing enabler stories.

As you know, below are the types of enabler stories.

1. Product Infrastructure – stories that directly support requested functional stories. 
This could include new and/or modified infrastructure. It might also identify refactoring 
opportunities but driven from the functional need.
2. Team Infrastructure – stories that support the team and their ability to deliver 
software. Often these surround tooling, testing, metrics, design, and planning. It could 
imply the team “creating” something or “buying and installing” something.
3. Refactoring – these are stories that identify areas that are refactoring candidates. Not 
only code needs refactoring, but also it can often include designs, automation, tooling, 
and any process documentation.
4. Bug Fixing – either clusters or packages of bugs that increase repair time or reduce 
aggregate of testing time. So this is more of an efficiency play.
5. Spikes – research stories that will result in learning, architecture & design, 
prototypes, and ultimately a set of stories and execution strategy that will meet the 
functional goal of the spike. Spikes need to err on the side of prototype code over 
documentation as well, although you don’t have to “demo” every spike.

You will be presented with a story and enabler type from a business analyst and you will be expected to make suggestions and recommendations on if the story type selected is correct. 
If the story type is correct, you will respond only with "No change".

If the story type is not correct, you will speak directly back to the business analyst in the imperitive tense, and explain the reasoning. 
You will use words like "may" and "might" to avoid offending the business analyst.
You will NOT speak in the first person. You will NOT ask questions.
'''

In [104]:

req_text = '''Story: 
In order to provide the parts analyst with the correct part source vendor 
code to be used in SPOs and PDC parts forecasting, 
the development team must add the NetSuite (NS) Vendor code to the vendor data 
that is being brought into the proxy and send both the NS Vendor Code and the MF Vendor Code to P&F

NetSuite Vendor Code sourced from VendorAccountNumber in DACH
Vendor Name should be updated to be sourced from VendorLegalName in DACH
Every MF Vendor code will have a NS Vendor code
Vendors added to new ERP system will NOT have a corresponding MF vendor code, so use the NS Vendor code
MF Vendor code will be passed in the LOC column on the LOCATION file
NS Vendor code will be passed in the U_VEN_ACC_CD column on the LOCATION FILE

Enabler Type:
Spike
'''

response = client.call_chat(messages = [ {"role": "system","content": p},
                    {"role": "user","content": req_text}
                ], seed = None)

response

'The enabler type selected may not be the most appropriate one. This task seems to be more about extending or modifying an integration between two systems, rather than researching or learning. It involves adding new data fields, ensuring that data is populated correctly and passed to other systems. Hence, a more appropriate enabler type could be Product Infrastructure.'

In [105]:
old_requirement_validation_sys = '''
Act as a member of the business analysis team to validate that the requested story meets the following requirements.

Story Requirement:
* A Story is defined as an informal explanation of a software feature written from the perspective of an end user.
The purpose of a story is to articulate piece of work, why it is important, and how the work will be implemented. 

Qualities of a story:
* A story must offer all the necessary information to make test development possible.
* A story should express what needs to be done in plain language, where possible, and in technical language when necessary.

To evaluate the story, score the story as a panel of five judges to evaluate if the story 
meets both the Story Requirement and Qualities. 

Judges, please comment on the nature of any disagreements, contradictary core ideas, or different perspectives on the same information. 
Be critical and explicit on any differences.

Scoring Scale of 0 to 5:
0, does not meet the Story Requirement or Qualities of a story.
5, meets the Story Requirement and has all the Qualities of a story.

The response should be in JSON format, as follows:
{
    "Judge 1" : "Score - Rationale"
    "Judge 2" : "Score - Rationale"
    "Judge #" : "Score - Rationale"
    "Overall Score" : The overall score goes here, just the number value.
    "Suggestions" : [] List specific suggestions to the author on how to improve the story. ex. Consider...
    "Updated Story" : Take the input and create an improved story.
}
'''

In [157]:
user_story_val_sys = '''
Act as a member of the business analysis team to validate that the requested user story meets the following requirements.

User Story Requirement:
* A user story is defined as an informal explanation of a software feature written from the perspective of an end user.
* The purpose of a story is to articulate piece of work and why it is important.

Qualities of a User Story:
* A user story must offer all the necessary requirements to make development possible.
* A user story should express what needs to be done in plain language, where possible, and in technical language when necessary.
* A user story should NEVER express the "how", only the "what" and "why".
* The story should NOT describe the approach to take, how to complete the requirement, or how the work will be done.

To evaluate the story, score the user story as a panel of five judges to evaluate if the story 
meets both the User Story Requirement and Qualities. 

Judges, please comment on the nature of any disagreements, contradictary core ideas, or different perspectives on the same information. 
Be critical and explicit on any differences.

Scoring Scale of 0 to 5:
0, does not meet the User Story Requirement or Qualities of a story.
5, meets the User Story Requirement and has all the Qualities of a story.

The response should be in JSON format, as follows:
{
    "Judge 1" : "Score - Rationale"
    "Judge 2" : "Score - Rationale"
    "Judge #" : "Score - Rationale"
    "Overall Score" : The overall score goes here, just the number value.
    "Suggestions" : [] List specific suggestions to the author on how to improve the user story. ex. Consider...
    "Updated Story" : Take the input and create an improved story.
}
'''

user_story_val_user = '''
Validate the following user story:
{story}
'''

In [151]:
enabler_story_val_sys = '''
Act as a member of the business analysis team to validate that the requested enabler story meets the following requirements.

Enabler Story Requirement:
* An enabler story (a.k.a. technical stories) are stories the team uses to develop new architecture and infrastructure needed to implement new user stories or improve previous work. In this case, the story may not directly touch any end user.
* The purpose of a story is to describe the requirements for a piece of work and why it is important.

Qualities of an Enabler Story:
* A enabler story should express what needs to be done in plain language, where possible, and in technical language when necessary.
* A enabler story should NEVER express the "how", only the "what" and "why".
* The story should NOT describe the approach to take, how to complete the requirement, or how the work will be done.

To evaluate the story, score the enabler story as a panel of five judges to evaluate if the story 
meets both the Enabler Story Requirement and Qualities. 

Judges, please comment on the nature of any disagreements, contradictary core ideas, or different perspectives on the same information. 
Be critical and explicit on any differences.

Scoring Scale of 0 to 5:
0, does not meet the Enabler Story Requirement or Qualities of a story.
5, meets the Enabler Story Requirement and has all the Qualities of a story.

The response should be in JSON format, as follows:
{
    "Judge 1" : "Score - Rationale"
    "Judge 2" : "Score - Rationale"
    "Judge #" : "Score - Rationale"
    "Overall Score" : The overall score goes here, just the number value.
    "Suggestions" : [] List specific suggestions to the author on how to improve the enabler story. ex. Consider...
    "Updated Story" : Take the input and create an improved story.
}
'''

enabler_story_val_user = '''
Validate the following enabler story:
{story}
'''

In [158]:
req_text = '''In order to provide the parts analyst with the correct part source vendor 
code to be used in SPOs and PDC parts forecasting, 
the development team must add the NetSuite (NS) Vendor code to the vendor data 
that is being brought into the proxy and send both the NS Vendor Code and the MF Vendor Code to P&F.
Use previous code in the repo to add the vendor code to the data.

NetSuite Vendor Code sourced from VendorAccountNumber in DACH
Vendor Name should be updated to be sourced from VendorLegalName in DACH
Every MF Vendor code will have a NS Vendor code
Vendors added to new ERP system will NOT have a corresponding MF vendor code, so use the NS Vendor code
MF Vendor code will be passed in the LOC column on the LOCATION file
NS Vendor code will be passed in the U_VEN_ACC_CD column on the LOCATION FILE'''

In [166]:


import json_repair

decoded_object = json_repair.loads(response)

In [168]:
decoded_object

{'Judge 1': '3 - The story provides clear technical requirements for adding the NetSuite vendor code to the existing vendor data. However, it is not written in plain language and can be difficult for non-technical stakeholders to understand. The story does not provide information on why adding a NetSuite vendor code is important.',
 'Judge 2': '4 - The story provides clear technical requirements for adding the NetSuite vendor code to the existing vendor data. However, it could be improved by including more detail around how much time and complexity will be involved to complete this work. Additionally, the story does not provide information on why adding a NetSuite vendor code is important.',
 'Judge 3': '3 - The story is very technically-focused and difficult for non-technical stakeholders to understand. It is missing key information around why it is important to add the NetSuite vendor code and why the MF Vendor code is no londer reliable enough for parts prediction.',
 'Judge 4': '4 

In [167]:
response = client.call_chat(messages = [ {"role": "system","content": old_requirement_validation_sys},
                    {"role": "user","content": requirement_validation_user.format(story = req_text)}
                ], seed = None)decoded_object

print(json.loads(response)['Suggestions'])

response = client.call_chat(messages = [ {"role": "system","content": requirement_validation_sys},
                    {"role": "user","content": requirement_validation_user.format(story = req_text)}
                ], seed = None)

print(json.loads(response)['Suggestions'])

SyntaxError: invalid syntax (1003804945.py, line 3)

In [156]:
response = client.call_chat(messages = [ {"role": "system","content": enabler_story_val_sys},
                    {"role": "user","content": enabler_story_val_user.format(story = req_text)}
                ], seed = None)

print(json.loads(response)['Suggestions'])
print(json.loads(response)['Overall Score'])

['Consider rephrasing some parts of the story to improve the clarity of the requirements and avoid describing how to complete the work.', 'Clarify the source of the Vendor Name for the first time.', 'Focus more on the business value of the enabler story.']
4
